In [ ]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random
import csv
from keras.callbacks import Callback
from keras.callbacks import CSVLogger
from sklearn.metrics import precision_recall_fscore_support

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Train/'
test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/'

num_classes = 7
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

cnn_model = tf.keras.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_model.add(tf.keras.layers.Flatten())

# RNN model
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((width, height * channels), input_shape=(height, width, channels)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),
    tf.keras.layers.BatchNormalization()
])

# Combine CNN and RNN models
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((1, height, width, channels), input_shape=(height, width, channels)),
    tf.keras.layers.TimeDistributed(cnn_model),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu', return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, activation='relu')),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'precision', 'recall', 'f1_score'])

epochs = 200
metrics_data = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

    # Get metrics from history
    accuracy = history.history['accuracy'][0]
    precision = history.history['precision'][0]
    recall = history.history['recall'][0]

    f1_score_tensor = history.history['f1_score'][0]
    f1_score_value = np.mean(f1_score_tensor)

    # Store metrics in a list
    metrics_data.append([accuracy, precision, recall, f1_score_value])

# Save metrics to a CSV file
csv_filename = 'training_metrics.csv'

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    writer.writerows(metrics_data)

print(f"Training metrics saved to {csv_filename}")

loss, accuracy, precision, recall, f1_score = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1 Score:", f1_score)

model.save('al-emadi_crnn_standardized_unknown.keras')

Found 3088 images belonging to 7 classes.
Found 768 images belonging to 7 classes.
Epoch 1/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 84s 679ms/step - accuracy: 0.2324 - f1_score: 0.1582 - loss: 2.0650 - precision: 0.3978 - recall: 0.0844
Epoch 2/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 70s 679ms/step - accuracy: 0.5598 - f1_score: 0.3916 - loss: 1.2438 - precision: 0.7617 - recall: 0.3715
Epoch 3/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 67s 655ms/step - accuracy: 0.7097 - f1_score: 0.6171 - loss: 0.8901 - precision: 0.8422 - recall: 0.5616
Epoch 4/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 73s 712ms/step - accuracy: 0.7444 - f1_score: 0.6567 - loss: 0.7208 - precision: 0.8488 - recall: 0.6426
Epoch 5/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 70s 678ms/step - accuracy: 0.8109 - f1_score: 0.7547 - loss: 0.5715 - precision: 0.8654 - recall: 0.7311
Epoch 6/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 68s 663ms/step - accuracy: 0.8319 - f1_score: 0.7840 - loss: 0.5217 - precision: 0.8864 - recall: 0.7679
Epoch 7/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 76s 729ms/step

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 7
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

batch_size = 32

model = tf.keras.models.load_model('al-emadi_crnn_standardized_unknown.keras')

test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/'

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

loss, accuracy, precision, recall, f1_score = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1 Score:", f1_score)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Print classification report (includes precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 768 images belonging to 7 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 16s 551ms/step - accuracy: 0.9794 - f1_score: 0.6631 - loss: 0.0969 - precision: 0.9811 - recall: 0.9794
Test Loss: 0.07280539721250534
Test Accuracy: 0.9817708134651184
Test Precision: 0.9830508232116699
Test Recall: 0.9817708134651184
Test F1 Score: tf.Tensor(
[0.98076916 1.         0.9383885  1.         1.         0.9845288
 1.        ], shape=(7,), dtype=float32)
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 514ms/step

Confusion Matrix:
[[102   0   2   0   0   0   0]
 [  0  52   0   0   0   0   0]
 [  1   0  99   0   0   4   0]
 [  0   0   0  50   0   0   0]
 [  0   0   0   0  50   0   0]
 [  1   0   6   0   0 350   0]
 [  0   0   0   0   0   0  51]]

Accuracy for each class:
Bebop: 0.9808
Crazyflie: 1.0000
Mambo: 0.9519
Matrice100: 1.0000
Mavic: 1.0000
NonDrone: 0.9804
Phantom: 1.0000

Classification Report:
              precision    recall  f1-score   support

       Bebop       0.98      0.98      0.98       104
   Crazyflie       1.00      1.00      1.00        52

In [ ]:
import os
import numpy as np
from scipy.stats import entropy
from tensorflow.keras.preprocessing import image

model = tf.keras.models.load_model('/content/al-emadi_crnn_standardized_unknown.keras')
classes = ['Bebop', 'Crazyflie', 'Mambo', 'Matrice100', 'Mavic', 'NonDrone', 'Phantom']

known_counter = 0
unknown_counter = 0

def load_spectrogram(file_path):
    # Load and preprocess the image
    num_classes = 7
    height = 128
    n_fft = 2048
    hop_length = 512
    sr = 16000
    T = 1

    hop_size_sec = hop_length / sr
    audio_length_samples = T * sr

    width = 1 + int((audio_length_samples - n_fft) / hop_length)

    channels = 3

    img = image.load_img(file_path, target_size=(height, width))  # Assuming input size expected by your model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    #img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Function to classify a single spectrogram
def classify_spectrogram(spectrogram, entropy_threshold):
    global known_counter
    global unknown_counter

    probabilities = model.predict(spectrogram)[0]  # Get class probabilities from the model
    ent = entropy(probabilities, base=2)  # Calculate entropy of the probability distribution

    if ent > entropy_threshold:
        unknown_counter+=1
        return "UnknownDrone"
    else:
        known_counter+=1
        max_index = np.argmax(probabilities)  # Get the index of the maximum probability
        return classes[max_index]  # Return the corresponding drone class

# Directory containing spectrogram files
unknown_spectrogram_dir = '/content/drive/MyDrive/UnknownDrone/'
spectrogram_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/Phantom/'

# Entropy threshold
entropy_threshold =  0.0175  # Adjust this threshold based on your model and validation results

# List all files in the directory
spectrogram_files = os.listdir(spectrogram_dir)

# Process each spectrogram file
for filename in spectrogram_files:
    if filename.endswith('.png'):  # Assuming spectrograms are saved as PNG files
        # Load the spectrogram (you need to implement this part based on your data loading method)
        spectrogram = load_spectrogram(os.path.join(spectrogram_dir, filename))

        # Classify the spectrogram
        predicted_class = classify_spectrogram(spectrogram, entropy_threshold)

        # Print the filename and predicted class
        print(f"{filename}: {predicted_class}")

print("Unknown Drone Count:", unknown_counter)
print("Known Drone Count:", known_counter)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
Phantom_Test_File1_0_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Phantom_Test_File2_3_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Phantom_Test_File1_8_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Phantom_Test_File2_13_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Phantom_Test_File2_10_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Phantom_Test_File2_16_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Phantom_Test_File2_24_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Phantom_Test_File2_21_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Phantom_Test_File3_4_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Phantom_Test_File1_0_PS-2_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Phantom_Test_File3_9_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Phantom_Test_File1_8_PS-2_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Phantom_Test_File2_3_PS-2_mel.png: